# Assignments
In this week's assignments, you will train a LightGBM regression model to predict public bike sharing demand given the attributes like datetime and weather conditions. The dataset ("bike_sharing_demand.csv", located under the same directory as this notebook) used in this week's assignments is a preprocessed version of [this kaggle dataset](https://www.kaggle.com/competitions/bike-sharing-demand/overview). You'll learn more about data preprocessing next week. Additionally, you'll use MLflow to track the model training and Deepchecks to evaluate the trained model. 

**Guidelines of submitting assignments**:
- For each assignment, a code skeleton is provided. Please put your solutions in between the `### START CODE HERE` and `### END CODE HERE` code comments. 
- Some assignments also require you to answer questions or capture screenshots in order to earn points. Please put all your answers and the required screenshots into a single PDF file. For each answer or screenshot, please clearly indicate which assignment it corresponds to in your PDF file. 
- When preparing your submission, be sure to include your assignment notebook with code cell outputs. It's important that these outputs are current and reflect the latest state of your code, as your grades may depend on them. Additionally, please include the PDF file that contains your answers and screenshots in your submission. 

In case type hints are new to you, you'll see something below in some code skeletons:
```python
def greeting(name: str) -> str:
    return 'Hello ' + name  
```
The annotation `name: str` means the parameter `name` is expected to be of type `str` and `-> str` means the type of the returned value is also `str`. These type hints are provided to help you understand the function's input requirements and its expected output in the assignments.

## Assignment 0: Set up the course environment (4 points)
You can earn 4 points for successfully setting up the course environment. To do so, simply write "yes" in your PDF file. Additionally, feel free to provide suggestions for improving the instructions within your PDF file.

In [ ]:
import pandas as pd
from lightgbm import LGBMRegressor
import mlflow
from deepchecks.tabular import Suite
from deepchecks.tabular.checks import TrainTestPerformance, ModelInferenceTime, MultiModelPerformanceReport
from deepchecks.tabular import Dataset
from pathlib import Path
import os
import pickle

from typing import Tuple

## Assignment 1: Download and split the dataset (2 points)

### 1a) Load the data
First, implement the `pull_data` function that loads a CSV as a Pandas DataFrame from a given location. 

In [ ]:
def pull_data(dataset_path: Path) -> pd.DataFrame:
    """
    Download the data set from a given path
    Args: 
        dataset_path (Path): Path of the CSV 
    Returns:
        A Pandas DataFrame of the dataset
    """
    ### START CODE HERE
    
    ### END CODE HERE


In [ ]:
# You can use this code cell to check if your pull_data function works correctly
dataset_path = Path.cwd() / "bike_sharing_demand.csv"
df = pull_data(dataset_path)
df.info()

<details>
    <summary>Expected output</summary>
    <img src="./images/dataset-info.png"/>
</details>

Below is the explanation of each column in the dataset:

**Variables**:

| Column name |  Explanation | type |
|-------------|---------------|----|
| season      | 1 = spring, 2 = summer, 3 = fall, 4 = winter | integer
| holiday     | whether the day is considered a holiday | integer
| workingday  | 1 if day is neither weekend nor holiday, otherwise 0. | integer
| weather     | 1: Clear, Few clouds, Partly cloudy, Partly cloudy; 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist; 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds; 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog | integer
| temp        | temperature in Celsius | float
| atemp       | "feels like" temperature in Celsius | float
| humidity    | relative humidity | integer
| windspeed   | wind speed | float
| hour        | the hours of the datetime| integer
| day         | the day of the datetime| integer
| month       | the month of the datetime| integer

**Targets**: 

| Column name | Explanation                                     | Type
|-------------|-------------------------------------------------| ----|
| count       | number of total rentals                         | integer

### 1b) Split the data into train and test DataFrames
Then implement the `splitData` function that splits the dataset into a training and a test dataset, using the last 168 rows of the dataset as the test data.

Hint: You may find [pandas.DataFrame.iloc](https://pandas.pydata.org/pandas-docs/version/1.5/reference/api/pandas.DataFrame.iloc.html) useful. 

In [ ]:
def splitData(input_df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Split a DataFrame into training and testing sets
    Args:
        input_df (DataFrame): The DataFrame to be splitted
    Return:
        A tuple of training and testing DataFrame
    """
    ### START CODE HERE
    
    ### END CODE HERE

In [ ]:
df = pull_data(dataset_path)
train, test = splitData(df)
train.describe()

<details>
    <summary>Expected output</summary>
    <img src="./images/train-df.png"/>
</details>

In [ ]:
print(f"The dimension of the training dataset should be {train.shape}")
print(f"The dimension of the testing dataset should be {test.shape}")

Expected output:

The dimension of the training dataset should be (10718, 12)

The dimension of the testing dataset should be (168, 12)

In [ ]:
# Split the training and testing DataFrames into features and targets
target = "count"
input_df = pull_data(Path.cwd() / "bike_sharing_demand.csv")
train, test = splitData(input_df)
train_x = train.drop([target], axis=1)
test_x = test.drop([target], axis=1)
train_y = train[[target]]
test_y = test[[target]]

## Assignment 2: Offline model evaluation using Deepchecks (2 points)

### 2a) Construct the Dataset objects used by Deepchecks
First, let's construct the Deepchecks Dataset objects from the train and testing DataFrames.

In [ ]:
# Categorical features, remember to specify the categorical features when you construct the datasets handled by Deepchecks
# See https://docs.deepchecks.com/stable/tabular/usage_guides/dataset_object.html
cat_features = ["season", "holiday", "workingday", "weather", "hour", "day", "month"]

### START CODE HERE
# Replace None with correct arguments
train_dataset = Dataset(None)
test_dataset = Dataset(None)
### END CODE HERE

### 2b) Deepchecks Suite with conditions
Implement the `evaluate` function that uses Deepchecks Suite to perform the following two tests:
1) Evaluate the model's MAE and RMSE on both training and testing dataset. This test should fail if the MAE or RMSE drops more than 20% on the testing dataset compared to the training dataset;
2) Evaluate the model's inference time. This test should fail if the inference time exceeds 0.1 second. 

Additionally, this function should save the evaluation results into an HTML file under the same directory as this notebook. 

**Hints**:
- [How to add conditions to a test?](https://docs.deepchecks.com/stable/general/usage/customizations/auto_examples/plot_configure_check_conditions.html)
- [Train test performance](https://docs.deepchecks.com/stable/api/generated/deepchecks.tabular.checks.model_evaluation.TrainTestPerformance.html)
- [Model inference time](https://docs.deepchecks.com/stable/tabular/auto_checks/model_evaluation/plot_model_inference_time.html)
- [Condition for comparing model performance between training and testing dataset](https://docs.deepchecks.com/stable/api/generated/deepchecks.tabular.checks.model_evaluation.TrainTestPerformance.add_condition_train_test_relative_degradation_less_than.html#deepchecks.tabular.checks.model_evaluation.TrainTestPerformance.add_condition_train_test_relative_degradation_less_than)
- [Condition for validating inference time](https://docs.deepchecks.com/stable/api/generated/deepchecks.tabular.checks.model_evaluation.ModelInferenceTime.add_condition_inference_time_less_than.html)
- [How to export the evaluation result into a file?](https://docs.deepchecks.com/stable/general/usage/export_save_results.html#save-result-as-an-html-report-save-as-html)

In [ ]:
def evaluate(train_dataset: Dataset, test_dataset: Dataset, model: LGBMRegressor, filename: str) -> str:
    """
    Use Deepchecks to evaluate 1) model's MAE and RMSE on both training and testing dataset, 2) model's inference time. Then save the evaluation results to an HTML file
    Args:
        train_dataset (Dataset): training Dataset
        test_dataset (Dataset): testing Dataset
        model (LGBMRegressor): The LightGBM regression model to be evaluated
        filename: The file name of the evaluation results
    Return:
        Name of the created HTML file
    """
    
    ### START CODE HERE
    
    ### END CODE HERE


In [ ]:
# We provide a testing model trained on the same bike demand dataset to help you check if your evaluate function works correctly
test_model = pickle.load(open("test-model.pkl", "rb"))
result_file_name = evaluate(train_dataset, test_dataset, test_model, "test-result.html")
print(result_file_name)

After running the above code cell, you should see a file named "test-result.html" appear under the same directory with this notebook.

Let's render this HTML file in a browser. 

In [ ]:
import webbrowser
webbrowser.open("file://" + str(Path.cwd() / result_file_name))

<details>
    <summary> Expected output (in the browser) </summary>
    <br />
    The test of MAE/RMSE should fail:
    <br />
    <img src="./images/deepchecks-train-test-performance.png">
    <br />
    The test of inference time should pass:
    <br />
    <img src="./images/deepchecks-inference-time.png">
</details>

### Screenshots to be submitted for Assignment 2
Similar to the expected output above, please submit screenshots of the web page showing which test passed and which test failed. 

## Assignment 3: Tracking model training in MLflow (2 points)
Similar to what you see in the MLflow tutorial, you need to complete the following tasks in this assignment: 
1) Use LightGBM to train a regression model to predict the bike sharing demand. The model should be trained using the training DataFrame you prepared previously and with the following hyperparameters: num_leaves = 63, learning_rate = 0.05, random_state = 42 (for reproducibility).
2) Use MLflow to track the model training. Specifically, you need to log the used hyperparameters and use the `evaluation` function you created above to evaluate the trained model. You also need to upload the Deepchecks evaluation result file and register the trained model to MLflow.  

**Hint**: You can check [LightGBM documentation on how to train a regression model](https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMRegressor.html#) if you're not familiar with it.

**Note**: LightGBM provides two types of APIs, the native APIs and the Scikit-learn (sklearn) APIs. The sklearn APIs can be seen as an interface that allows you to use a LightGBM model as if it was trained using the sklearn library. The model trained in this assignment will be used in other assignments in the upcoming weeks. Please use the **sklearn API** in this assignment to avoid unexpected issues.

In [ ]:
# mlflow configuration
MLFLOW_S3_ENDPOINT_URL = "http://mlflow-minio.local"
MLFLOW_TRACKING_URI = "http://mlflow-server.local"
AWS_ACCESS_KEY_ID = "minioadmin"
AWS_SECRET_ACCESS_KEY = "minioadmin"
mlflow_experiment_name = "week1-lgbm-bike-demand"

os.environ["MLFLOW_S3_ENDPOINT_URL"] = MLFLOW_S3_ENDPOINT_URL
os.environ["AWS_ACCESS_KEY_ID"] = AWS_ACCESS_KEY_ID
os.environ["AWS_SECRET_ACCESS_KEY"] = AWS_SECRET_ACCESS_KEY
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(mlflow_experiment_name)

# model hyperparameters
num_leaves = 63
learning_rate = 0.05
random_state = 42

In [ ]:
model = LGBMRegressor(num_leaves=num_leaves, learning_rate=learning_rate, random_state=random_state)

with mlflow.start_run() as run:
    ### START CODE HERE

    ### END CODE HERE



### Screenshots to be submitted for Assignment 3
To get the points from Assignment 3, please submit the following screenshots:
1. The logs of your MLflow run. Please include the parameters of the model in your screenshot. 
<details>
    <summary>Example</summary>
    <img src="./images/mlflow-run.png" width=1000>
</details>

2. The details of the MLflow run including uploaded Deepchecks evaluation result file;
<details>
    <summary>Example</summary>
    <img src="./images/mlflow-run-detail1.png" width=1000>
    <img src="./images/mlflow-run-detail2.png" width=1000>
    <img src="./images/mlflow-run-detail3.png" width=1000>
</details>

3. The registered model.
<details>
    <summary>Example</summary>
    <img src="./images/mlflow-model.png" width=1000>
</details>

## Assignment 4: Evaluate the trained model against another model (2 points)
Suppose your colleague had trained an ElasticNet model for the same use case of bike sharing demand prediction. In this assignment, your tasks are:
1. Using Deepchecks to compare MAE and RMSE of your LightGBM model to the ElasticNet model and saving the result as an HTML file;
1. Uploading the result file to MLflow, the file should be under the MLflow Run where you trained your LightGBM model in Assignment 3. E.g.,

<img src="./images/ass4-example.png" width=300 />

Note that the idea here is to attach a file to an existing MLflow Run, not to create a new MLflow Run and then upload the file under the new MLflow Run. 

You may find the following docs helpful: 
- [Multi model performance report](https://docs.deepchecks.com/stable/tabular/auto_checks/model_evaluation/plot_multi_model_performance_report.html).
- [mlflow.start_run](https://mlflow.org/docs/2.3.2/python_api/mlflow.html#mlflow.start_run) (Pay attention to the use of the `run_id` parameter).

In [ ]:
# Load the ElasticNet model first
old_model = pickle.load(open("old-model.pkl", "rb"))
old_model

In [ ]:
# Use Deepchecks to compare your LightGBM model to the ElasticNet model and save the result to an HTML file
### START CODE HERE

### END CODE HERE

In [ ]:
# Upload the result file to MLflow. The file should be under the MLflow Run where you trained your LightGBM model
### START CODE HERE

### END CODE HERE

### Screenshots to be submitted for Assignment 4
The details of the MLflow run including uploaded Deepchecks model comparison result file.
<details>
    <summary>Example</summary>
    <img src="./images/deepchecks-compare-models.png" />
</details>